# Clipper de Áudio (YouTube) — WoW PT-BR

Use este notebook para **baixar um trecho curto de áudio** de um vídeo do YouTube (ex.: uma fala) usando **timestamp** das legendas.

**Como usar (atores/diretores):**
1. Rode o bloco ① **Instalar dependências** (só 1x por sessão).
2. No bloco ②, **preencha a URL/ID, tempo inicial e duração** (ou cole uma *Especificação de Clipe* como `https://youtu.be/ID?t=123 | 5s`).
3. Clique ▶ **no bloco ②**. O notebook baixa o áudio, recorta e **baixa o `.mp3`**.

Dicas de tempo:
- Em **Tempo inicial**, use `83`, `1:23`, `00:01:23` ou `1m23s`. Se a URL tiver `?t=...`, será usada.
- **Duração** típica: 3–6 segundos.


In [ ]:
# @title ① Instalar dependências (rodar 1x por sessão)
# @markdown Este bloco instala `yt-dlp` e verifica `ffmpeg`.
import sys, subprocess, shutil

def run(cmd):
  p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
  print(p.stdout)
  return p.returncode

print("Instalando yt-dlp...")
run([sys.executable, "-m", "pip", "install", "-q", "yt-dlp"])

if not shutil.which("ffmpeg"):
  print("ffmpeg não encontrado; instalando (pode levar 1 min)...")
  run(["apt-get", "update", "-y"])
  run(["apt-get", "install", "-y", "ffmpeg"])
else:
  print("ffmpeg OK")

print("Pronto.")

In [ ]:
# @title ② Gerar clipe MP3
# @markdown Preencha os campos abaixo e clique ▶ neste bloco.
from typing import Optional
import re, os, json, tempfile, subprocess, uuid
from yt_dlp import YoutubeDL

# --- Formulário Colab ---
especificacao = "" # @param {type:"string"}
youtube_url_or_id = "" # @param {type:"string"}
tempo_inicial = "0"    # @param {type:"string"}
duracao_seg = 5        # @param {type:"number"}
bitrate_kbps = 96      # @param {type:"number"}

# exemplo de "especificacao":  https://youtu.be/ID?t=123 | 5s

# --- Utilidades ---
def parse_time_any(s: str) -> float:
  """Converte '1:23', '00:01:23', '83', '1m23s' -> segundos (float). Também lê ?t=123 de uma URL."""
  s = (s or "").strip().lower()
  if not s:
    return 0.0
  m = re.search(r"[?&#]t=(\d+)", s)
  if m:
    try: return float(m.group(1))
    except: pass
  if re.search(r"\d+h|\d+m|\d+s", s):
    h = m = sec = 0
    mh = re.search(r"(\d+)h", s)
    mm = re.search(r"(\d+)m", s)
    ms = re.search(r"(\d+)s", s)
    if mh: h = int(mh.group(1))
    if mm: m = int(mm.group(1))
    if ms: sec = int(ms.group(1))
    return float(h*3600 + m*60 + sec)
  if ":" in s:
    parts = [p for p in s.split(":") if p!=""]
    try:
      parts = [float(p) for p in parts]
      if len(parts)==3: return parts[0]*3600 + parts[1]*60 + parts[2]
      if len(parts)==2: return parts[0]*60 + parts[1]
    except: pass
  try:
    return float(s)
  except:
    return 0.0

def parse_duration_any(s: str) -> float:
  s = (s or "").strip().lower()
  if not s: return 0.0
  m = re.search(r"(\d+)\s*s", s)
  if m: return float(m.group(1))
  try: return float(s)
  except: return 0.0

def extract_video_id(url_or_id: str) -> Optional[str]:
  s = (url_or_id or "").strip()
  if not s:
    return None
  m = re.search(r"(?:v=|youtu\.be/|youtube\.com/embed/)([A-Za-z0-9_-]{6,})", s)
  if m: return m.group(1)
  if re.fullmatch(r"[A-Za-z0-9_-]{6,}", s):
    return s
  return None

# --- Extrair dados da especificacao (se presente) ---
spec_id = extract_video_id(especificacao)
spec_t  = parse_time_any(especificacao)
spec_d  = parse_duration_any(especificacao)

# Aplicar: valores do formulário têm prioridade; senão, usar os da especificação, senão defaults.
vid = extract_video_id(youtube_url_or_id) or spec_id
start_sec = parse_time_any(tempo_inicial) or spec_t or 0.0
try:
  dur = float(duracao_seg) if duracao_seg else (spec_d or 5.0)
except:
  dur = spec_d or 5.0

if not vid:
  print("❗ Forneça uma URL do YouTube/ID válido OU cole uma Especificação de Clipe contendo o link.")
  raise SystemExit

start_sec = max(0.0, start_sec)
dur = max(0.1, dur)

print(f"Vídeo: {vid} | Início: {start_sec:.3f}s | Duração: {dur:.3f}s | Bitrate: {int(bitrate_kbps)}k")

# --- Baixar melhor áudio ---
work = tempfile.mkdtemp(prefix="clipper_")
outtmpl = os.path.join(work, "%(id)s.%(ext)s")
ydl_opts = {
    "skip_download": False,
    "format": "bestaudio/best",
    "outtmpl": outtmpl,
    "quiet": True,
    "no_warnings": True,
    "noprogress": True,
}
print("Baixando áudio do YouTube...")
src_path = None
with YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(vid, download=True)
    cand = os.path.join(work, f"{info['id']}")
    for ext in ("m4a","webm","opus","mp4","mkv","mp3"):
        p = cand + "." + ext
        if os.path.exists(p):
            src_path = p
            break
if not src_path:
    print("❗ Não foi possível localizar o arquivo de áudio baixado.")
    raise SystemExit

# --- Recortar com ffmpeg ---
out_name = f"clip_{vid}_{int(start_sec)}s_{int(dur)}s_{uuid.uuid4().hex[:6]}.mp3"
out_path = os.path.join("/content", out_name)
print("Gerando MP3 recortado...")

cmd = [
    "ffmpeg", "-y",
    "-ss", str(start_sec),
    "-t", str(dur),
    "-i", src_path,
    "-vn",
    "-acodec", "libmp3lame",
    "-b:a", f"{int(bitrate_kbps)}k",
    "-ar", "44100",
    out_path
]
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
if proc.stdout: print(proc.stdout.splitlines()[-1])

if not os.path.exists(out_path):
    print("❗ Falha ao criar o MP3.")
    raise SystemExit

print(f"✅ Pronto: {out_path}")
print(f"Link YouTube no ponto: https://youtu.be/{vid}?t={int(start_sec)}")

# --- Download automático ---
try:
  from google.colab import files
  files.download(out_path)
except Exception as e:
  print("Observação: se o download não abrir, use o painel 'Files' à esquerda para salvar o arquivo.")